In [71]:
traning_data =[
    ['Green',3,'Mango'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon'],
]

In [99]:
traning_data

[['Green', 3, 'Mango'],
 ['Yellow', 3, 'Mango'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [100]:
header = ["color","diameter","label"]

In [101]:
def unique_vals(rows,col):
  # unique values
  return set([row[col] for row in rows])

# unique_vals(traning_data,1)

In [102]:
def class_counts(rows):
  # Counts the number of each type pf example in dataset
  counts = {}
  for row in rows:
    label = row[-1]
    if label not in counts:
        counts[label]=0
    counts[label] +=1
  return counts

In [103]:
def is_numeric(value):
  # Test if a avlue is Numric
  return isinstance(value,int) or isinstance(value,float)

In [105]:


class Question:
    # A Question is used to partition a dataset.
    # This class just records a 'column number' (e.g. 0 for color) and a
    # 'column value' (e.g Green). This 'match' method is used to compare
    # the feature value in an example to the feature value stored in
    # the question. See the demo below.
    def __init__(self, column, value):
        self.column = column
        self.value = value
    def match(self,example):
      # Compare the Feature value in an example
      # feature value in this question
      val = example[self.column]
      if is_numeric(val):
        return val >= self.value
      else:
        return val == self.value
    def __repr__(self):
      #This is just a helper method to print
      #the questation in a readable formate
      condition="=="
      if is_numeric(self.value):
        condition = ">="
      return "Is %s %s %s?" %(
          header[self.column],condition,str(self.value)
      )



In [106]:
def partition(rows,question):
  ture_rows,false_rows=[],[]
  for row in rows:
    if question.match(row):
        ture_rows.append(row)
    else:
         false_rows.append(row)
  return ture_rows,false_rows
     #Partitions a dataset.
     #For each row in the dataset, check if it matches the question. If so, add it to 'true rows, otherwise, add it to 'false rows".


In [79]:
# Demo:
# Let's partition the training data based on whether rows are Red. I
#true rows, false rows = partition(training data, Question(e, 'Red'))
#This will contain all the 'Red' rows.
# true_rows
# This will contain everything else.
#false_rows

""Calculate the Gini Impurity for a list of rows. There are a few different ways to do this, I thought this one was the most concise. See: https://en.wikipedia.org/wiki/Decision tree learning#Gini_impurity

In [107]:
def gini(rows):
  counts = class_counts(rows)
  impurity=1
  for lbl in counts:
      prob_of_lbl = counts[lbl]/float(len(rows))
      impurity -= prob_of_lbl**2
  return impurity

"Information Gain. I The uncertainty of the starting node, minus the weighted impurity of two child nodes.

Demo:
Calculate the uncertainy of our training data. current uncertainty gini(training data)

How much information do we gain by partioning on 'Green'?
true rows, false rows = partition(training data, Question(e, Green')) info gain(true rows, false rows, current uncertainty)
What about if we partioned on 'Red' instead?
true rows, false rows = partition(training data, Question (0, 'Red'))  info_gain(true rows, false rows, current_uncertainty)

In [108]:
def info_gain(left,right,current_uncertanity):
    p=float(len(left))/(len(left)+len(right))
    return current_uncertanity - p * gini(left)-(1-p)*gini(right)

Find the best question to ask by iterating over every feature / value and calculating the information gain

In [82]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])

        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)

            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [83]:
class Leaf:
  #"A Leaf node classifies data.
  #This holds a dictionary of class (e.g., "Mango") -> number of times
  #it appears in the rows from the training data that reach this leaf.
  def __init__(self,rows):
    self.predictions=class_counts(rows)
class Decision_Node:
  #A Decision Node asks a question.
  #This holds a reference to the question, and to the two child nodes.
  def __init__(self,question,true_branch,flase_branch):
    self.question=question
    self.true_branch=true_branch
    self.flase_branch =flase_branch


Builds the tree
Try partitioing the dataset on each of the unique attribute calculate the information gain and return the question that produces the highest gain.

In [109]:


def build_tree(rows):
    # Try partitioning the dataset on each of the unique attributes,
    # calculate the information gain, and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain.
    # Since we can ask no further questions, we'll return a Leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Decision_Node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow depending on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [115]:
def print_tree(node , spacing=""):
  # World's most elegant tree printing funtion
    if isinstance(node,Leaf):
         print(spacing+"Predict",node.predictions)
         return

    print(spacing+str(node.question))
  #call this function recursively on the true branch print (spacing+ --> True:')
    print(spacing+'---> True')
    print_tree(node.true_branch,spacing+" ")
  #call this function recursively on the false branch print (spacing+ --> False:') print tree(node.false branch, spacing + " " ")
    print(spacing +'--> False')
    print_tree(node.flase_branch,spacing+" ")

In [120]:
def classify(row,node):
  # Base case : we've reached a leaf
  if isinstance(node,Leaf):
    return node.predictions
  # Decide Wheather to follow the true-branch or the false-branch
  # Compare the feayure / value stored in the node,
  # tp the example we're considering
  if node.question.match(row):
    return classify(row,node.true_branch)
  else:
    return classify(row,node.flase_branch)

In [122]:
def print_leaf(counts):
# print the predictions at a leaf
  total = sum(counts.values())*1.0
  probs={}
  for lbl in counts.keys():
    probs[lbl]=str(int(counts[lbl]/total*100))+"%"
  return probs

In [123]:
if __name__ == '__main__':
    my_tree = build_tree(traning_data)
    print_tree(my_tree)
    testing_data = [
        ['Green', 3, 'Mango'],
        ['Yellow', 4, 'Mango'],
        ['Red', 2, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
    ]
    for row in testing_data:
        print("Actual: %s. Predicted: %s" % (row[-1], print_leaf(classify(row, my_tree))))


Is diameter >= 3?
---> True
 Is color == Yellow?
 ---> True
  Predict {'Mango': 1, 'Lemon': 1}
 --> False
  Predict {'Mango': 1}
--> False
 Predict {'Grape': 2}
Actual: Mango. Predicted: {'Mango': '100%'}
Actual: Mango. Predicted: {'Mango': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Mango': '50%', 'Lemon': '50%'}


In [114]:
if __name__ == '__main__':
 my_tree = build_tree(traning_data)
    print_tree(my_tree)

IndentationError: ignored